In [3]:
!pip install openpyxl

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


# trials segmentation notebook to trim the videos and update the datasets based on the trial start and trial ends times in the csv

In [157]:
import pandas as pd
import numpy as np
import os
import imageio_ffmpeg
import subprocess as sp

In [90]:
base_path = r"C:\Users\aleja\Box\Awake Project\Maze data\simplermaze\mouse 6357\raw_cross_check"


session_paths= []

for i in os.listdir(base_path):
    if ".csv" in i:
        session_path = os.path.join(base_path, i)
        session_paths.append(session_path)

sessions_ID = ["3.6", "3.7", "3.8"]

sessions_path_dict = dict(zip(sessions_ID, session_paths))

# # for i, j in sessions.items():
# #     print (i, j)


print(sessions_path_dict)#
print(session_paths)


{'3.6': 'C:\\Users\\aleja\\Box\\Awake Project\\Maze data\\simplermaze\\mouse 6357\\raw_cross_check\\session3.6.csv', '3.7': 'C:\\Users\\aleja\\Box\\Awake Project\\Maze data\\simplermaze\\mouse 6357\\raw_cross_check\\session3.7.csv', '3.8': 'C:\\Users\\aleja\\Box\\Awake Project\\Maze data\\simplermaze\\mouse 6357\\raw_cross_check\\session3.8.csv'}
['C:\\Users\\aleja\\Box\\Awake Project\\Maze data\\simplermaze\\mouse 6357\\raw_cross_check\\session3.6.csv', 'C:\\Users\\aleja\\Box\\Awake Project\\Maze data\\simplermaze\\mouse 6357\\raw_cross_check\\session3.7.csv', 'C:\\Users\\aleja\\Box\\Awake Project\\Maze data\\simplermaze\\mouse 6357\\raw_cross_check\\session3.8.csv']


In [ ]:
def make_csvs_no_empty_rows(session_dict):

    for i, j in session_dict.items():
        print(j)

        miao = pd.read_excel(j)
        miao = miao.dropna(how="all")

        miao.to_csv(base_path + f"/session{i}.csv", index = False)


# make_csvs_no_empty_rows(sessions_path_dict)

C:\Users\aleja\Box\Awake Project\Maze data\simplermaze\mouse 6357\raw_cross_check\mouse6357session3.6_data_2024-08-28_11_58_14.xlsx
C:\Users\aleja\Box\Awake Project\Maze data\simplermaze\mouse 6357\raw_cross_check\mouse6357session3.7_data_2024-08-29_10_23_02.xlsx
C:\Users\aleja\Box\Awake Project\Maze data\simplermaze\mouse 6357\raw_cross_check\mouse6357session3.8_data_2024-08-30_10_07_55.xlsx


In [91]:
mouse_6357_path = r"C:\Users\aleja\Box\Awake Project\Maze data\simplermaze\mouse 6357"

exp_session_paths = []
for exp_session in os.listdir(mouse_6357_path):
    for i in sessions_ID:
        if i in exp_session:
            # print (exp_session)
            exp_session_path = os.path.join(mouse_6357_path, exp_session)
            print(exp_session_path)
            exp_session_paths.append(exp_session_path)

# print(exp_session_paths)

C:\Users\aleja\Box\Awake Project\Maze data\simplermaze\mouse 6357\2024-08-28_11_58_146357session3.6
C:\Users\aleja\Box\Awake Project\Maze data\simplermaze\mouse 6357\2024-08-29_10_23_026357session3.7
C:\Users\aleja\Box\Awake Project\Maze data\simplermaze\mouse 6357\2024-08-30_10_07_556357session3.8


In [155]:
def trim_video_into_segments(parent_path, trials_csv_path, session):

    

    ffmpeg_path = imageio_ffmpeg.get_ffmpeg_exe()
    #make output directory
    output_dir = os.path.join(parent_path, "new_segments\\")
    os.makedirs(output_dir, exist_ok=True)

    # look for the .mp4 movie in the parent path

    for i in os.listdir(parent_path):
        if ".mp4" in i:
            video = os.path.join(parent_path, i)

    trial_df = pd.read_csv(trials_csv_path)

    trial_df["trial_ID"]= [f"trial_{i:03}".format(i) for i in range(len(trial_df))]
    trial_df["state_sequence"] = [0 for i in range(len(trial_df)) ]
    trial_df["video_path"] = [f"{output_dir}trial_{i:03}.mp4".format(i) for i in range(len(trial_df))]
    trial_df["trial_start_time_s"]= (trial_df["trial_start_time"] / 1000)
    trial_df["end_trial_time_s"] = (trial_df["end_trial_time"]/ 1000)
    trial_df["duration_s"] = trial_df["end_trial_time_s"] - trial_df["trial_start_time_s"]
    trial_df["start_trial_frame"] = round(trial_df["trial_start_time_s"]*30)
    trial_df["end_trial_frame"] = round(trial_df["end_trial_time_s"]*30)

    
    trial_df.to_csv(parent_path + f"/new_session{session}_trials.csv", index = False)
    print("saved df to csv")

    # trial_df.head(12)

    for index, row in trial_df.iterrows():
        # print(index)

        start_time = row["trial_start_time_s"]  # Start at 0 seconds
        end_time = row["end_trial_time_s"]    # End at 7 seconds
        target_path = row["video_path"]
        
        print(f" trial{index}\nExtracting from {start_time:.2f}s to {end_time:.2f}s")
        # ffmpeg_extract_subclip(video, start_time, end_time, target_path)

        # sp.call([ffmpeg_path, '-loglevel', 'quiet', '-ss', str(start_time), '-to', str(end_time), '-i', video, '-c', 'copy', f"{target_path}"])


In [ ]:
# print(exp_session_paths)
# print(sessions_path_dict)
# print(sessions_ID)

old_trial_csv_paths = []
new_trial_csv_paths = []
# print(len(exp_session_paths))

for session in sessions_ID:
    for parent_path in exp_session_paths:
        if session in parent_path:
            # print(parent_path)
            # trials_csv_path =  sessions_path_dict[session]
            
            # print(f"session: {session}, parent path: {parent_path}, trials_csv_path: {trials_csv_path}")
            # trim_video_into_segments(parent_path, trials_csv_path, session)

            for files in os.listdir(parent_path):

                if "new_session" in files:
                    # print(parent_path +"\\"+files)
                    new_csv = parent_path +"\\"+ files
                    new_trial_csv_paths.append(new_csv)

                if "trial_info.csv" in files and "clean" not in files: 
                    old_csv = parent_path +"\\"+ files
                    old_trial_csv_paths.append(old_csv)




print(new_trial_csv_paths)
print(old_trial_csv_paths)
            

['C:\\Users\\aleja\\Box\\Awake Project\\Maze data\\simplermaze\\mouse 6357\\2024-08-28_11_58_146357session3.6\\new_session3.6_trials.csv', 'C:\\Users\\aleja\\Box\\Awake Project\\Maze data\\simplermaze\\mouse 6357\\2024-08-29_10_23_026357session3.7\\new_session3.7_trials.csv', 'C:\\Users\\aleja\\Box\\Awake Project\\Maze data\\simplermaze\\mouse 6357\\2024-08-30_10_07_556357session3.8\\new_session3.8_trials.csv']
['C:\\Users\\aleja\\Box\\Awake Project\\Maze data\\simplermaze\\mouse 6357\\2024-08-28_11_58_146357session3.6\\mouse6357_session3.6_trial_info.csv', 'C:\\Users\\aleja\\Box\\Awake Project\\Maze data\\simplermaze\\mouse 6357\\2024-08-29_10_23_026357session3.7\\mouse6357_session3.7_trial_info.csv', 'C:\\Users\\aleja\\Box\\Awake Project\\Maze data\\simplermaze\\mouse 6357\\2024-08-30_10_07_556357session3.8\\mouse6357_session3.8_trial_info.csv']


In [162]:
new_file = new_trial_csv_paths[1]
# old_file = r"C:/Users/aleja/Box/Awake Project/Maze data/simplermaze/mouse 6357/2024-08-28_11_58_146357session3.6/trials_with_entropy_prova_corrected_frames.csv"

new_df= pd.read_csv(new_file)
# old_df = pd.read_csv(old_trial_csv_paths[0])




# old_entropy_3_6_df = pd.read_csv(old_file, encoding='cp1252')
old_entropy_3_7_df  =pd.read_csv( r"C:\Users\aleja\Box\Awake Project\Maze data\simplermaze\mouse 6357\2024-08-29_10_23_026357session3.7\trials_corrected_final_frames.csv")
# # # print(new_df.columns)
# # # print(len(old_df))
# # print(old_entropy_3_7_df.columns)

# # # finding the rows where "trials_start_time" and "end_trials_time"

counter = 0

for o_index, o_row in old_entropy_3_7_df.iterrows():

    o_start_time = o_row["trial_start_time"]
    o_end_time = o_row["end_trial_time"]
    o_state_sequence = o_row["state_sequence"]

    for n_index, n_row in new_df.iterrows():
        n_start_time = n_row["trial_start_time"]
        n_end_time = n_row["end_trial_time"]
        n_state_sequence = n_row["state_sequence"]#

        if o_start_time == n_start_time and o_end_time == n_end_time:
            counter+=1
            print(f"{counter} \nold start:{o_start_time}, new start: {n_start_time}\n old end:{o_end_time}, new end: {n_end_time}\n  ")

            new_df.at[n_index, "state_sequence"] = o_state_sequence



new_df.to_csv(new_file, index=False)







1 
old start:3, new start: 3.0
 old end:33697, new end: 33697.0
  
2 
old start:33712, new start: 33712.0
 old end:62884, new end: 62884.0
  
3 
old start:62887, new start: 62887.0
 old end:87381, new end: 87381.0
  
4 
old start:87387, new start: 87387.0
 old end:139814, new end: 139814.0
  
5 
old start:139825, new start: 139825.0
 old end:186086, new end: 186086.0
  
6 
old start:186102, new start: 186102.0
 old end:221840, new end: 221840.0
  
7 
old start:221852, new start: 221852.0
 old end:251570, new end: 251570.0
  
8 
old start:251585, new start: 251585.0
 old end:314985, new end: 314985.0
  
9 
old start:315002, new start: 315002.0
 old end:332486, new end: 332486.0
  
10 
old start:332502, new start: 332502.0
 old end:461495, new end: 461495.0
  
11 
old start:461502, new start: 461502.0
 old end:549769, new end: 549769.0
  
12 
old start:549785, new start: 549785.0
 old end:600032, new end: 600032.0
  
13 
old start:600034, new start: 600034.0
 old end:621466, new end: 621

C:\Users\aleja\AppData\Local\Temp\ipykernel_17036\1492040411.py:35: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '1' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  new_df.at[n_index, "state_sequence"] = o_state_sequence


In [ ]:
sesh36 = pd.read_csv(new_trial_csv_paths[0])

# total_duration = 0
# for i in sesh36['duration_s']:
#     total_duration += i

# print(total_duration/60)

def get_min_from_ms(ms):
    return (ms/ 1000)/60

def get_min_from_frame(frame):
    return (frame/30)/60


three7_first_nest = get_min_from_ms(2949451)
three6_first_nest = get_min_from_ms(2618884)

print(three6_first_nest)
print(three7_first_nest)

print((152258/30)/60)



43.648066666666665
49.157516666666666
84.58777777777777
